(tmux_custom-environment)=
# Создание настраиваемых рабочих окружений в tmux

Чаще всего разработчику при работе над проектом, приходится запускать целый набор различных инструментов и программ.
Например, веб-разработчику, скорее всего, потребуется командная оболочка, текстовый редактор, консоль базы данных и окно
для запуска автоматизированного набора тестов для разрабатываемого приложения.
Для всего этого необходимо множество окон и требуется вводить множество команд, чтобы все это запустить.

Было бы гораздо удобнее иметь возможность запускать все эти программы, каждую в своей панели или окне, в одном сеансе
tmux, используя всего одну команду.
Этого можно добиться, используя клиент-серверную модель tmux для создания пользовательских сценариев, которые создают
среды разработки, разделяя окна и автоматически запуская в них необходимые программы.
Сначала рассмотрим, как сделать это вручную, а затем рассмотрим более продвинутые автоматические инструменты.

## Создание пользовательских настроек с помощью команд tmux

Помимо создания новых сеансов, команда `tmux` имеет множество других параметров.
Например, можно разделить окна на панели, изменять макеты или даже запускать приложения в рамках сеанса.

Ключом к этому является опция `-t` или «target».
Если существует именованный сеанс tmux, к нему можно подключиться вот так:
```console
$ tmux attach -t [session_name]
```

Можно использовать этот указатель, чтобы применить команду tmux к нужному сеансу tmux.

Создадим новый сеанс tmux под названием «development»:
```console
$ tmux attach -s development
```

Отсоединимся от сеанса с помощью `PREFIX` `d`.

Можно разделить окно горизонтали, даже в неактивном сеансе tmux по, выполнив эту команду:
```console
$ tmux split-window -h -t development
```

Присоединитесь к сеансу еще раз, чтобы убедиться, что команда была выполнена:
```console
$ tmux attach -t development
```

На самом деле не нужно даже отключаться от сеанса tmux для отправки команд.
Можно просто открыть другой терминал и снова разделить окно, но на этот раз вертикально.
Откроем второе окно или вкладку терминала и введем следующую команду:
```console
$ tmux split-window -v -t development
```

Используя этот подход, можно легко настроить свое окружение.
Давайте рассмотрим эту концепцию, создав нашу собственную среду разработки.

## Сценарий конфигурации проекта

В разделе {ref}`«Изучение основ» <tmux_basic>` были описаны команды tmux `new-session` и `new-window`.
Давайте, используя эти и подобные команды, напишем простой скрипт который запустит новый сеанс tmux, создаст основное
окно с парой панелей и два дополнительных окна с одной панелью в каждом.
И напоследок, запустит отдельное приложение в каждой из панелей.

Начнем с создания нового скрипта под названием `development` в домашнем каталоге.
Сделаем этот скрипт исполняемым файлом, чтобы можно было запустить его, как и исполняемую программу в терминале.
Для этого выполним эти команды в терминале:
```console
$ touch ~/development
$ chmod +x ~/development
```

Пусть после запуска сеанса tmux переходит в каталог для проекта.
Назовем этот каталог `devproject`.
Но прежде чем перейти в этот каталог, лучше сначала его создать.

```console
$ mkdir ~/devproject
```

Теперь откроем скрипт `~/development` в текстовом редакторе и добавим команду, создающую новый сеанс tmux с именем "development":
Передадим пару дополнительных параметров при создании этого нового сеанса.
Во-первых, присвоим ему имя с помощью опции `-s`.
Во-вторых, назначим окну имя «editor».
И, наконец, немедленно отсоединяемся от этого нового сеанса с помощью опции `-d`.
```
# ~/development

tmux new-session -s development -n editor -d
```

Затем, используя команду tmux `send-keys`, изменим текущий каталог на каталог проекта:
```
# ~/development

tmux send-keys -t development 'cd ~/devproject' C-m
```
Последовательность `C-m` в конце строки имитирует последовательность возврата каретки, представленную как
{bdg-dark}`Ctrl`-{bdg-dark}`M`.
Так «заставит» tmux нажать клавишу {bdg-dark}`Enter` после ввода команды.

Используем тот же подход, чтобы открыть текстовый редактор Vim в этом окне:
```
# ~/development

tmux send-keys -t development 'vim' C-m
```

С помощью этих трех команд мы создали новый сеанс, перешли в каталог и открыли текстовый редактор, но наша среда еще не
настроена до конца.
Давайте разделим главное окно пополам по горизонтали «editor», чтобы у нас было небольшое окно терминала внизу.
Сделаем это с помощью команды `split-window `:
```
# ~/development

`tmux split-window -v -t development
```

Можно было бы указать процент, используя что-то вроде
```
# ~/development

tmux split-window -v -p 10 -t development
```

Но пока просто оставим команду `split-window` как есть, а затем выберем один из макетов tmux по умолчанию ---
`main-horizontal`:
```
# ~/development

tmux select-layout -t development main-horizontal
```

## Выбор панелей и окон

Итак, мы создали первое окно и разделили его на две панели, причем нужно, чтобы нижняя панель должна открываться в папке проекта.
Давайте рассмотрим как выполнять команды в определенных панелях и окнах.


С помощью таких команд, как `send-keys`, можно указать не только целевой сеанс, но и целевое окно и панель.
В файле конфигурации, который был создан в разделе {ref}`«Настройка tmux» <tmux_configuring>`, была переопределена нумерация окон и панелей с единицы.
В нашем случае у нас есть две панели в текущем окне:
```
|-------------------------|
|                         |
|         Панель 1        |
|                         |
|-------------------------|
|                         |
|         Панель 2        |
|                         |
|-------------------------|
```

В панели 1 открыт текстовый редактор Vim. В панель 2 необходимо отправить команду, которая выполнит переход в каталог проекта.
Выбор панели осуществляется с помощью следующего формата `[сеанс]:[окно].[панель]`.
Таким образом для выбора панели 2 необходимо использовать ` development:1.2`.
Итак, добавьте эту строку в свой скрипт, и вы получите именно то, что нужно:
```
# ~/development

tmux send-keys -t development:1.2 'cd ~/devproject' C-m
```

## Создание и выбор окон

Давайте завершим эту настройку, добавив в сеанс еще пару окон.
Нам нужно второе окно в нашем сеансе, которое будет полноэкранной консолью.
Новое окно можно создать с помощью команды `new-window `:
```
# ~/development

tmux new-window -n console -t development
tmux send-keys -t development:2 'cd ~/devproject' C-m
```

После создания окна, используем `send-keys`, чтобы снова перейти в каталог нашего проекта.
Так как имеется только одна панель во вновь созданном окне, нужно указать только номер окна.

После запуска сеанса, необходимо чтобы отображалось первое окно, добиться этого можно с помощью команды `select-window`:
```
# ~/development

tmux select-window -t development:1
tmux attach -t development
```

Таким образом, можно продолжать добавлять к этому сценарию новую функциональность, создавая дополнительные окна и
панели, запуская удаленные подключения к серверам, отслеживая файлы журналов, подключаясь к консолям базы данных или
даже выполняя команды, которые загружают последнюю версию кода из репозитория.
Но остановимся на этом и просто завершим сценарий, окончательно подключившись к сеансу, чтобы он появился на экране,
готовый к началу работы.

Полностью сценарий выглядит следующим образом:
```
# ~/development

tmux new-session -s development -n editor -d
tmux send-keys -t development 'cd ~/devproject' C-m
tmux send-keys -t development 'vim' C-m
tmux split-window -v -t development
tmux select-layout -t development main-horizontal
tmux send-keys -t development:1.2 'cd ~/devproject' C-m
tmux new-window -n console -t development
tmux send-keys -t development:2 'cd ~/devproject' C-m
tmux select-window -t development:1
tmux attach -t development
```

Запустим его с помощью команды
```console
$ ~/development
```

Открывшееся окно будет так:

![](./img/-060-018.png)


Одним из недостатков такого подхода является то, что этот сценарий каждый раз создает новый сеанс.
Это создаст проблемы, если запустить его во второй раз, когда сеанс «development» уже запущен.

Можно изменить сценарий, чтобы проверять, существует ли сеанс с таким именем, используя команду tmux `has-session`,
и создать новый сеанс только в том случае, если его нет, например, так:

```
# ~/development

tmux has-session -t development
if [$? != 0]
then
  tmux new-session -s development -n editor -d
  tmux send-keys -t development 'cd ~/devproject' C-m
  tmux send-keys -t development 'vim' C-m
  tmux split-window -v -t development
  tmux select-layout -t development main-horizontal
  tmux send-keys -t development:1.2 'cd ~/devproject' C-m
  tmux new-window -n console -t development
  tmux send-keys -t development:2 'cd ~/devproject' C-m
  tmux select-window -t development:1
fi
tmux attach -t development
```

Такой подход отлично работает для настройки одного проекта.

Можно сделать его более универсальным, используя переменную для имени проекта, но давайте рассмотрим пару других
способов настройки для управления несколькими проектами.

##  Использование конфигураций для настройки tmux

Сам файл `~.tmux.conf` может включать команды, устанавливающие окружение по умолчанию.
Если необходимо каждый запускать tmux в одной и той же папке по умолчанию или автоматически разделять окна,
можно прописать это прямо в конфигурацию по умолчанию, просто используя соответствующие команды.

Также имеется возможность указать файл конфигурации при запуске экземпляра tmux, используя флаг `-f`.
Таким образом, не нужно менять исходный файл конфигурации по умолчанию, и можно для каждого проекта использовать
отдельный файл конфигурации с собственными параметрами, такими как сочетания клавиш для запуска команд или запуска
набора тестов.

Давайте сделаем это. Создадим новый файл с именем app.conf:
```console
$ touch app.conf
```

Внутри этого файла можно использовать те же самые команды, что и в предыдущем разделе, но поскольку это файл
конфигурации, а не сценарий оболочки, не нужно явно добавлять префикс к каждой команде tmux:
Добавьте этот код в ваш файл app.conf:

```
# app.conf

source-file ~/.tmux.conf
new-session -s development -n editor -d
send-keys -t development 'cd ~/devproject' C-m
send-keys -t development 'vim' C-m
split-window -v -t development
select-layout -t development main-horizontal
send-keys -t development:1.2 'cd ~/devproject' C-m
new-window -n console -t development
send-keys -t development:2 'cd ~/devproject' C-m
select-window -t development:1
```

Этот код сначала загружает существующий файл .tmux.conf, поэтому будут сохранены все настройки среды, которые
в нем были определены, включая сочетания клавиш и настройки строки состояния.
Это не обязательно, но если этого не сделать, то все сочетания клавиш и параметры вернутся к значениям по умолчанию,
или придется заново переопределять их в этом файле.

Чтобы использовать этот файл конфигурации, необходимо передать флаг `-f`, за которым следует путь к файлу конфигурации.
Также нужно запускать tmux с помощью команды `attach`, например:
```console
$ tmux -f app.conf attach
```

Это связано с тем, что по умолчанию tmux всегда вызывает команду `new-session` при запуске.
Приведенный файл уже создает новый сеанс, поэтому, если не использовать `attach`, будет запущено два сеанса.